<a href="https://colab.research.google.com/github/Martagilant/ONLINE_DS_THEBRIDGE_MartaGilAntunano/blob/main/18_Practica_Obligatoria_Scraping_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image-2.png](attachment:image-2.png)

## PRACTICA OBLIGATORIA: SQL

* La práctica obligatoria de esta unidad consiste en extraer datos de películas de dos fuentes diferentes y cruzarlos, para una fuente tiraremos de Scraping y para la otra de API, y luego practicarás con pandas un poco. Descarga este notebook en tu ordenador y trabaja en local. Ten en cuenta que tendrás que descar los directorios de imágenes y datos adicionales, si los hubiera.
* Recuerda que debes subirla a tu repositorio personal antes de la sesión en vivo para que puntúe adecuadamente.  
* Recuerda también que no es necesario que esté perfecta, sólo es necesario que se vea el esfuerzo.
* Esta práctica se resolverá en la sesión en vivo correspondiente y la solución se publicará en el repo del curso.

## Objetivo

El objetivo final es tener un `DataFrame` con una serie de películas de 2023, sus datos más destacados (titulo, año de lanzamiento, director, guionistas, actores, género(s), coste de producción *budget* y recaudación a nivel USA y a nivel mundial). Para ello tendrás que acceder a dos fuentes de información y unir los datos, además cada fuente con un método diferente. Al final tendrás que contestar a unas preguntas sencillas.


## #0

Como siempre, haz aquí todos los imports que necesites:

In [9]:
import pandas as pd
import requests
import certifi
import ssl


## #1 API: The Movie Database (TMDb)

Cambiamos el orden temático de la unidad y empezamos por obtener los datos de las películas antes de obtener sus datos de ingresos porque tiene más sentido, ya que mandan las películas.

Vamos a acceder a TMDb cuyos datos básicos te dejo aquí (pero recuerda que es la última vez, a partir de ahora tendrás que buscar la info de las APIs por tu cuenta)


### API de The Movie Database (TMDb)

- **URL Base**: `https://api.themoviedb.org/3`


#### Endpoints Principales:

1. **Obtener Detalles de una Película**:
   - **Endpoint**: `/movie/{movie_id}`
   - **Método**: GET
   - **Uso**: Obtiene detalles específicos de una película utilizando su ID.
   - **Ejemplo**: `https://api.themoviedb.org/3/movie/550?api_key=tu_api_key`


2. **Buscar Películas**:
   - **Endpoint**: `/search/movie`
   - **Método**: GET
   - **Uso**: Busca películas por título.
   - **Ejemplo**: `https://api.themoviedb.org/3/search/movie?query=Inception&api_key=tu_api_key`

3. **Buscar películas por diferentes criterios**:
   - **Endpoint**: `/discover/movie`
   - **Método**: GET
   - **Uso**: Obtiene información de las películas por criterios, tiene más de 30 filtros y funciones de ordenación.
   - **Ejemplo**: `https://api.themoviedb.org/3/person/500?api_key=tu_api_key`

4. **Obtener Películas Populares**:
   - **Endpoint**: `/movie/popular`
   - **Método**: GET
   - **Uso**: Devuelve una lista de películas populares.
   - **Ejemplo**: `https://api.themoviedb.org/3/movie/popular?api_key=tu_api_key`

Recuerda reemplazar `tu_api_key` con tu propia API key.

### 1.1

Investiga en la página de [documentación](https://developer.themoviedb.org/reference/discover-movie) detallada del endpoint de "Discover" e identifica los parámetros que tenemos que pasar a la API para obtener las 20 películas americanas y las 20 españolas de 2023 más populares.

Nota: Las respuestas de esta API están paginadas, y cada página contiene 20 respuestas, si quisieramos más tendríamos que incluir el parámetro `page` con su número en peticiones diferentes, por eso para esta práctica lo vamos a limitar a 20 respuestas, para que no haya que jugar con el paginado).

In [ ]:
#para peliculas 2023:
primary_release_year=2023
with_original_language=en/esp
sort_by=popularity.desc
page = 1

### 1.2

Para poder hacer la llamada a la API con endpoint "Discover" necesitamos pasarle una serie de parámetros como los anteriores y la API_Key. Hay dos formas de hacerlo, mediante la querystring en la url o mediante el argumento params de request. Como tenemos que hacer una consulta para las películas estadounidenses y otra para las españolas, vamos a utilizar los dos métodos.

En este ejercicio se pide usar el método de incluir todos los parámetros en la url mediante la sintaxis ?\<parametro>=\<valor>&... para obtener las 20 películas norteamericanas más populares de 2023, según TMDb. Para ello crea una variable `url_usa` a partir de la siguiente guía:  

url_usa = base_url + end_point + "?\<parametro1>=\<valor1>&\<parametros2>=\<valor2>..."  

Donde tendrá que incluir los parámetros "api_key" con valor tu api_key para TMDb y el resto de parámetros que necesites con sus valores para obtener los datos pedidos (utiliza `year` para el año). Luego utiliza el metodo request adecuado y vuelca la salida en `respuesta_usa`

Ojo el valor del nombre de Estados Unidos en este caso no es USA, pero casi.

EXTRA: Hay una forma de que te devuelva los datos en español cuando corresponda, a ver si lo logras.

In [10]:
api_key = "a67f4ae851f17dface95243a7ab7d7e8"
base_url = "https://api.themoviedb.org/3/"
end_point = "discover/movie"
url_usa = (
    f"{base_url}{end_point}"
    f"?api_key={api_key}"
    f"&primary_release_year=2023"
    f"&with_origin_country=US"
    f"&sort_by=popularity.desc"
    f"&page=1")


#Para español solo habria que cambiar el idioma = language=es-ES

In [11]:
respuesta_usa = requests.get(url_usa)

### 1.3

Escribe el código de respuesta y su explicación. Vuelca los datos de la respuesta en una variable `datos_usa` (es decir el resultado del metodo `json`). Muestra los campos del json de respuesta y decide cuál es el que vamos a usar para extrarer los datos. Muestra los valores para ese campo y luego asignalo a la variable `pelis_usa`.

In [12]:
respuesta_usa.status_code #si funciona y no me da error

200

In [22]:
datos_usa = respuesta_usa.json()

datos_usa
print(datos_usa.keys())


dict_keys(['page', 'results', 'total_pages', 'total_results'])


In [25]:
print("\nTítulos de películas:")
for pelicula in datos_usa["results"]:
    print(pelicula["title"])

pelis_usa = datos_usa["results"]


Títulos de películas:
Cosmic Chaos
The Vigilante
Fast X
John Wick: Chapter 4
Spider-Man: Across the Spider-Verse
Oppenheimer
Barbie
The Super Mario Bros. Movie
Migration
Transformers: Rise of the Beasts
Mission: Impossible - Dead Reckoning Part One
Five Nights at Freddy's
Elemental
Poor Things
Guardians of the Galaxy Vol. 3
Anyone But You
The Equalizer 3
The Little Mermaid
Napoleon
Meg 2: The Trench


### 1.3 Alt

Si no tienes una API-Key o no has conseguido obtener los valores, lee el archivo "./data/usa_movies.json" de la forma conveniente y vuelca sus datos en una variable llamada `pelis_usa`

### 1.4

Convierte los resultados a un `DataFrame` aprovechando que es una lista de diccionarios con la misma estructura. Asigna lo a "df_movies". Muestra sus primeras filas y la info general.

In [29]:
df= pd.DataFrame(pelis_usa)
df.head(10)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/m2mzlsJjE3UAqeUB5fLUkpWg4Iq.jpg,"[53, 878]",1165067,en,Cosmic Chaos,"Battles in virtual reality, survival in a post...",260.8825,/mClzWv7gBqgXfjZXp49Enyoex1v.jpg,2023-08-03,Cosmic Chaos,False,6.900,27
1,False,/nYQJYdfoy7Tkx6bPzUpmCMmyJ4y.jpg,"[53, 28]",1151470,en,The Vigilante,"Returning from Afghanistan, Jessica, a Spec OP...",63.9038,/xD9LpYZYmNch2EhHWIJXXFayENH.jpg,2023-09-08,The Vigilante,False,5.900,20
2,False,/4XM8DUTQb3lhLemJC51Jx4a2EuA.jpg,"[28, 80, 53, 12, 9648]",385687,en,Fast X,Over many missions and against impossible odds...,33.0435,/fiVW06jE7z9YnO4trhaMEdclSiC.jpg,2023-05-17,Fast X,False,7.000,5833
3,False,/7I6VUdPj6tQECNHdviJkUHD2u89.jpg,"[28, 53, 80]",603692,en,John Wick: Chapter 4,"With the price on his head ever increasing, Jo...",29.3793,/vZloFAK7NmvMGKE7VkF5UHaz0I.jpg,2023-03-22,John Wick: Chapter 4,False,7.722,6948
4,False,/kVd3a9YeLGkoeR50jGEXM6EqseS.jpg,"[16, 28, 12, 878]",569094,en,Spider-Man: Across the Spider-Verse,"After reuniting with Gwen Stacy, Brooklyn’s fu...",29.2927,/8Vt6mWEReuy4Of61Lnj5Xj704m8.jpg,2023-05-31,Spider-Man: Across the Spider-Verse,False,8.345,7449
5,False,/neeNHeXjMF5fXoCJRsOmkNGC7q.jpg,"[18, 36]",872585,en,Oppenheimer,The story of J. Robert Oppenheimer's role in t...,27.1628,/8Gxv8gSFCU0XGDykEGv7zR1n2ua.jpg,2023-07-19,Oppenheimer,False,8.066,10049
6,False,/mbYTRO33LJAgpCMrIn9ibiWHbMH.jpg,"[35, 12]",346698,en,Barbie,Barbie and Ken are having the time of their li...,25.1240,/iuFNMS8U5cb6xfzi51Dbkovj7vM.jpg,2023-07-19,Barbie,False,6.981,9745
7,False,/9n2tJBplPbgR2ca05hS5CKXwP2c.jpg,"[10751, 35, 12, 16]",502356,en,The Super Mario Bros. Movie,"While working underground to fix a water main,...",24.7790,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,2023-04-05,The Super Mario Bros. Movie,False,7.624,9566
8,False,/gklkxY0veMajdCiGe6ggsh07VG2.jpg,"[16, 12, 35, 10751]",940551,en,Migration,After a migrating duck family alights on their...,21.5611,/apQL85BMRgkBWQq6pBXKOLfCyaV.jpg,2023-12-06,Migration,False,7.405,1966
9,False,/2vFuG6bWGyQUzYS9d69E5l85nIz.jpg,"[878, 12, 28]",667538,en,Transformers: Rise of the Beasts,When a new threat capable of destroying the en...,23.1988,/gPbM0MK8CP8A174rmUwGsADNYKD.jpg,2023-06-06,Transformers: Rise of the Beasts,False,7.244,4852


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              20 non-null     bool   
 1   backdrop_path      20 non-null     object 
 2   genre_ids          20 non-null     object 
 3   id                 20 non-null     int64  
 4   original_language  20 non-null     object 
 5   original_title     20 non-null     object 
 6   overview           20 non-null     object 
 7   popularity         20 non-null     float64
 8   poster_path        20 non-null     object 
 9   release_date       20 non-null     object 
 10  title              20 non-null     object 
 11  video              20 non-null     bool   
 12  vote_average       20 non-null     float64
 13  vote_count         20 non-null     int64  
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 2.0+ KB


### 1.5



De las columnas que tenemos realmente nos interesan unas pocas. Yo me voy a quedar con las siguientes:

```python
col_seleccionadas = ["id","genre_ids","original_title","title","overview","vote_average"]
```

Añade alguna más si quieres, pero manten las anteriores siempre. Construye un nuevo `DataFrame`, "df_pop_usa", que contenga sólo las columnas seleccionadas

In [35]:
col_seleccionadas = ["id","genre_ids","original_title","title","overview","vote_average"]
df_pop_usa = df[col_seleccionadas]
df_pop_usa.head(10)

,id,genre_ids,original_title,title,overview,vote_average
0,1165067,"[53, 878]",Cosmic Chaos,Cosmic Chaos,"Battles in virtual reality, survival in a post...",6.900
1,1151470,"[53, 28]",The Vigilante,The Vigilante,"Returning from Afghanistan, Jessica, a Spec OP...",5.900
2,385687,"[28, 80, 53, 12, 9648]",Fast X,Fast X,Over many missions and against impossible odds...,7.000
3,603692,"[28, 53, 80]",John Wick: Chapter 4,John Wick: Chapter 4,"With the price on his head ever increasing, Jo...",7.722
4,569094,"[16, 28, 12, 878]",Spider-Man: Across the Spider-Verse,Spider-Man: Across the Spider-Verse,"After reuniting with Gwen Stacy, Brooklyn’s fu...",8.345
5,872585,"[18, 36]",Oppenheimer,Oppenheimer,The story of J. Robert Oppenheimer's role in t...,8.066
6,346698,"[35, 12]",Barbie,Barbie,Barbie and Ken are having the time of their li...,6.981
7,502356,"[10751, 35, 12, 16]",The Super Mario Bros. Movie,The Super Mario Bros. Movie,"While working underground to fix a water main,...",7.624
8,940551,"[16, 12, 35, 10751]",Migration,Migration,After a migrating duck family alights on their...,7.405
9,667538,"[878, 12, 28]",Transformers: Rise of the Beasts,Transformers: Rise of the Beasts,When a new threat capable of destroying the en...,7.244


### 1.6

Guardamos ese dataframe para luego, ahora vamos a buscar las películas españolas. Para ello vamos a probar el otro método, ahora crea un diccionario siguiendo esta estructura:

```python
querystring = {
    "api_key": "PON AQUI TU API-KEY",
    "parametro1": "valor1",
    ...
}
```

In [36]:
querystring = {
    "api_key": api_key,
    "language": "es-ES",                      # Resultados estén en español
    "primary_release_year": 2023,             # Año de lanzamiento
    "with_origin_country": "ES",              # Código de país para España
    "sort_by": "popularity.desc",             # Ordenar
    "page": 1
}

### 1.7

Ahora construye la url_spain, pero ten en cuenta que ya sólo necesitas la url base y el endpoint


In [40]:
url_spain = base_url + end_point

### 1.8

Lanza la petición usando el método adecuado y con argumentos `url_spain` para la url y el diccionario creado en \#1.6 para el argumento "params" (que es como se envían parametros fuera de la url, hay otra forma a través del parámetro data, pero lo iremos viendo a lo largo del curso). Asigna la respuesta a `respuesta_spain`, y muestra su "status_code" y su "reason".

In [42]:
respuesta_spain = requests.request("GET", url_spain, params=querystring)

In [44]:
respuesta_spain.status_code

200

In [45]:
respuesta_spain.reason

'OK'

### 1.9
Repite todo el proceso que hicimos para las películas norteamericanas hasta obtener un dataframe similar (con las mismas columnas), llámalo `df_pop_spain`. Si no tienes el API-Key o no has sabido obtener los datos de la API, utiliza los datos que hay en "./data/spain_movies.json"

In [53]:
datos_spain = respuesta_spain.json()
print("\nTítulos de películas:")
for pelicula in datos_spain["results"]:
    print(pelicula["title"])

pelis_spain = datos_spain["results"]


Títulos de películas:
Culpa mía
Moscas
A través de mi ventana 2: A través del mar
La sociedad de la nieve
Robot Dreams
Nowhere
Malditas
Héroes de Central Park
Momias
La mesita del comedor
Todos los nombres de Dios
Tin y Tina
Extraña forma de vida
Momonsters: la película
El hombre del saco
Slasher
Creatura
La niña de la comunión
Bird Box Barcelona
Mi soledad tiene alas


In [55]:
df_spain= pd.DataFrame(pelis_spain)
df_spain.head(10)
col_seleccionadas = ["id","genre_ids","original_title","title","overview","vote_average"]
df_pop_spain = df_spain[col_seleccionadas]
df_pop_spain.head(10)

,id,genre_ids,original_title,title,overview,vote_average
0,1010581,"[10749, 18]",Culpa mía,Culpa mía,"Noah debe dejar su ciudad, novio y amigos para...",7.823
1,1177908,[53],Moscas,Moscas,El señor Machi es un empresario aficionado a l...,6.500
2,988078,"[10749, 18, 35]",A través del mar,A través de mi ventana 2: A través del mar,"Tras pasar un año separados, Raquel y Ares se ...",6.600
3,906126,"[18, 36]",La sociedad de la nieve,La sociedad de la nieve,"El 13 de octubre de 1972, el vuelo 571 de la F...",8.002
4,838240,"[16, 18, 35, 878]",Robot Dreams,Robot Dreams,DOG es un perro solitario que vive en Manhatta...,8.012
5,1151534,"[53, 18]",Nowhere,Nowhere,"Embarazada, sola y a la deriva en el mar, una ...",7.400
6,1154864,"[27, 28, 18]",Malditas,Malditas,Las vidas de seis niñas pertenecen a mundos di...,6.306
7,1059264,"[16, 12, 35, 10751]",The Inseparables,Héroes de Central Park,Un títere de Don Quijote fugitivo con una imag...,6.400
8,816904,"[16, 12, 14, 10751]",Momias,Momias,"En las entrañas de la tierra, ¡existe una ciud...",6.970
9,1056380,"[27, 53, 18, 35]",La mesita del comedor,La mesita del comedor,Una pareja que acaba de ser padres y no pasa p...,6.959


### 1.10
Para concluir esta parte, junta los dos dataframes en uno solo, df_base_movies, usando el método [pd.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) de pandas. Luego convierte la columna id en su índice.

In [58]:
df_movies = pd.concat([df_pop_usa, df_pop_spain], ignore_index= True)
df_movies = df_movies.set_index("id")
df_movies.head(40)


,genre_ids,original_title,title,overview,vote_average
id,,,,,
1165067,"[53, 878]",Cosmic Chaos,Cosmic Chaos,"Battles in virtual reality, survival in a post...",6.900
1151470,"[53, 28]",The Vigilante,The Vigilante,"Returning from Afghanistan, Jessica, a Spec OP...",5.900
385687,"[28, 80, 53, 12, 9648]",Fast X,Fast X,Over many missions and against impossible odds...,7.000
603692,"[28, 53, 80]",John Wick: Chapter 4,John Wick: Chapter 4,"With the price on his head ever increasing, Jo...",7.722
569094,"[16, 28, 12, 878]",Spider-Man: Across the Spider-Verse,Spider-Man: Across the Spider-Verse,"After reuniting with Gwen Stacy, Brooklyn’s fu...",8.345
872585,"[18, 36]",Oppenheimer,Oppenheimer,The story of J. Robert Oppenheimer's role in t...,8.066
346698,"[35, 12]",Barbie,Barbie,Barbie and Ken are having the time of their li...,6.981
502356,"[10751, 35, 12, 16]",The Super Mario Bros. Movie,The Super Mario Bros. Movie,"While working underground to fix a water main,...",7.624
940551,"[16, 12, 35, 10751]",Migration,Migration,After a migrating duck family alights on their...,7.405


## #2 Scrapping: BoxOfficeMojo

### 2.1 [EXTRA]
Si el primer ejercicio es extra, porque el tiempo de ejecución depende muy mucho de lo bien que se te de inspeccionar la página y entender los tags y los atributos. Pero te animo a que lo intentes, lo haremos en clase en cualquier caso.

Este ejercicio consiste en scrapear la página de ingresos mundiales de las películas con mayores ingresos que ofrece el site BoxOfficeMojo en la siguiente url:

In [ ]:
url_mojo = "https://www.boxofficemojo.com/year/2023/?area=ES"

Tendrás que acceder inspeccionar y buscar los tags necesarios para obtener una dataframe que refleje los mismos datos que muestra la tabla.

Para hacerte más sencillo el trabajo, te doy aquí algunas pistas:

* Los nombres de los campos se pueden obtener de los tag "th" que hay dentro del tag "table" con atributo `class='a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated'`

* Los valores se encuentran en los tags "tr" del tag "tbody". Tendrás que ver cómo sacarlos de ahí.

1. Inspecciona la página y ve desplegando hasta encontrar tbody y ahí ya investiga hasta dar con los tags últimos que tienen los datos.
2. Repasa las soluciones a los ejercicios del workout de WebScraping para conseguir llegar al `DataFrame`

### 2.2

Si no has podido hacer el ejercicio Extra anterior, lee los datos que hay en "./data/mojo_data.xlsx", cargándolos en un dataframe al que puedes poner por nombre "df_mojo". Usar "Rank" como índice y muestra el contenido y su información general.

In [59]:
df_mojo = pd.read_excel("./data/mojo_data.xlsx", index_col = "Rank")

FileNotFoundError: [Errno 2] No such file or directory: './data/mojo_data.xlsx'

### 2.3

Hora de volver a nuestro `DataFrame` de peliculas obtenido de la API. Pégale las recuadaciones de df_mojo, de forma que mantegamos las 40 peliculas populares, no importa cuantas de df_mojo queden. ¿Qué método vas a usar?¿Qué tipo de join quieres hacer?¿Qué campos son los índices?

### 2.4
Muestra los 10 primeros registros del resultado del cruce y su información general

### 2.5

Muestra las películas españolas para las que sí hay información de ingresos. Cuidado: Tiene "truco"


### 2.6
Para terminar, vuelve a cruzar las tablas pero ahora con un inner join de forma que sólo nos quedemos con las peliculas que tienen recaudación en box office. Antes marca de alguna manera el indice de popularidad en df_base_movies y después de hacer el join comprueba "visualmente" cuanto de correlados están los rakings de popularidad e ingresos.